Обучим логистическую регрессию для предсказания того, откликнется клиент на рекламное предложение (`target = 1`) или нет (`target = 0`).

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/clients_data.csv')

In [3]:
data.head()

,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_DLQ_NUM,TARGET
0,49,1,0,1,2,1,5000.0,1,1,2,0
1,32,1,0,1,3,3,12000.0,1,1,1,0
2,52,1,0,1,4,0,9000.0,2,1,0,0
3,39,1,0,1,1,1,25000.0,1,1,3,0
4,30,1,0,0,0,0,12000.0,2,1,2,0


In [9]:
data.shape

(15223, 11)

In [4]:
from sklearn.model_selection import train_test_split

X = data.drop('TARGET', axis=1)
y = data['TARGET']

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, random_state=123)

**Задание:** выведите на экран количество объектов каждого класса. Сколько процентов объектов относятся к положительному классу? Ответ округлите до целого числа (например, если доля объектов положительного класса равна 0.412, в ответ запишите 41, имея в виду 41 процент).

In [10]:
y[y == 0].shape[0]

13411

In [11]:
y[y == 1].shape[0]

1812

In [15]:
round(y[y == 0].shape[0]/y.shape[0], 2)*100

88.0

Обучим логистическую регрессию с параметрами по умолчанию.

In [16]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

In [17]:
lr.fit(Xtrain, ytrain)

prediction = lr.predict(Xtest)

In [22]:
prediction.shape

(4567,)

Метрику accuracy не стоит использовать при сильном дисбалансе классов. Поэтому посчитайте f1_score для оценки качества модели на тестовых данных.

f1_score принимает значения от 0 до 1. Чем ближе к 1, тем лучше модель.

In [19]:
from sklearn.metrics import f1_score

f1_score(ytest, prediction)

0.0

In [21]:
ytest

10164    0
13499    0
4091     0
3754     0
4427     0
        ..
13378    0
9295     1
13960    0
12774    0
13199    0
Name: TARGET, Length: 4567, dtype: int64

**Вопрос:** чему равен f1_score?

Удивительно, да?

Давайте разберемся, почему качество такое низкое.

Предскажем вероятности классов с помощью обученной логистической регрессии на тестовых данных.

In [23]:
probs_test = lr.predict_proba(Xtest)

probs_test[:10]

array([[0.86331972, 0.13668028],
       [0.86305957, 0.13694043],
       [0.80967732, 0.19032268],
       [0.73110167, 0.26889833],
       [0.87780414, 0.12219586],
       [0.96122864, 0.03877136],
       [0.95965491, 0.04034509],
       [0.96560659, 0.03439341],
       [0.92114871, 0.07885129],
       [0.9349701 , 0.0650299 ]])

По вероятностям видно, что вероятности отнесения к положительному классу очень низкие.

Попробуем изменить порог для перевода вероятности в классы.

**Вопрос:** чему равен f1_score, если все объекты с вероятностью не меньшей 0.1, относить к положительному классу?

Ответ округлите до сотых.

In [35]:
probs_test[:, 1]

array([0.13668028, 0.13694043, 0.19032268, ..., 0.09132017, 0.09727959,
       0.07518077])

In [63]:
probs_new = probs_test[:, 1] >= 0.1

In [64]:
probs_new

array([ True,  True,  True, ..., False, False, False])

In [65]:
f1_score(ytest, probs_new)

0.2469521725539231

У обученной модели можно посмотреть веса (как и в линейной регрессии). Выведем на экран веса модели (model.coef_, model.intercept_).

In [66]:
lr.coef_, lr.intercept_

(array([[-5.55214705e-02, -7.90868782e-04, -4.28172545e-04,
         -1.07248154e-03, -1.34958177e-03, -4.95807875e-04,
          1.13131722e-05, -1.80011616e-03, -1.26843809e-03,
         -1.45283567e-05]]), array([-0.00115086]))

Создайте pd.DataFrame, где в первом столбце стоят названия признаков, а во втором - их веса (так удобнее анализировать результат). Отсортируйте таблицу по убыванию весов.

**Вопрос:** какой признак имеет наибольший положительный вес?

In [76]:
name = Xtrain.columns.to_list()

In [83]:
coeffs = lr.coef_.flatten()

In [85]:
my_df = pd.DataFrame([name, coeffs])

In [88]:
my_df = my_df.T

In [92]:
my_df.sort_values(by=1, ascending=False)

,0,1
6,PERSONAL_INCOME,0.000011
9,LOAN_DLQ_NUM,-0.000015
2,SOCSTATUS_PENS_FL,-0.000428
5,DEPENDANTS,-0.000496
1,SOCSTATUS_WORK_FL,-0.000791
3,GENDER,-0.001072
8,LOAN_NUM_CLOSED,-0.001268
4,CHILD_TOTAL,-0.00135
7,LOAN_NUM_TOTAL,-0.0018
0,AGE,-0.055521


## Бонус 

**Задание 1:** подберите порог для перевода вероятностей в классы, дающий максимальное значение f1_score.

In [ ]:
# ваш код здесь

**Задание 2:** во вложенном цикле подберите одновременно коэффициент регуляризации C у логистической регрессии и порог для перевода вероятностей в классы, дающие максимальное значение f1_score.

По-хорошему, чтобы не переобучиться, эти величины надо подбирать не по тесту, а по отдельной выборке.

Поэтому разобъем данные изначально на три части: Xtrain, Xval, Xtest.

*   В цикле при подборе С и порога будем обучаться по Xtrain, а предсказывать и измерять качество по Xval.

*   Качество итоговой модели с найденными C и порогом измерьте по Xtest. 

Так не переобучимся!


In [ ]:
Xtrain_new, Xval, ytrain_new, yval = train_test_split(Xtrain, ytrain, train_size=0.7, random_state=123)

# ваш код для подбора C и порога здесь

In [ ]:
model = LogisticRegression(C = ...)

model.fit(Xtrain, ytrain) # обучаемся на всех тренировочных данных

prediction = model.predict_proba(Xtest)

classes = ... # переведите полученные вероятности в классы по найденному порогу

In [ ]:
# вычислите значение f1_score на тестовых данных